In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylin

In [3]:
import torch
import timm

# Assuming your model is saved as 'model.pth'
model_path = '/content/drive/MyDrive/Major_Project_Final-Yr/vit_feature_pretrained.pth'
model = timm.create_model('vit_base_patch16_224', pretrained=False)  # Create model
model.load_state_dict(torch.load(model_path))  # Load saved state
model.eval().to() # Set to evaluation mode


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [4]:
from PIL import Image
import torchvision.transforms as transforms

def load_image_and_extract_features(image_name_without_extension):
    image_path = f'/content/drive/MyDrive/Major_Project_Final-Yr/off_image_test/{image_name_without_extension}.bmp'
    image = Image.open(image_path).convert('RGB')

    # Define your transformations here (similar to when you extracted features)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Apply transformations
    image_tensor = transform(image).unsqueeze(0).to()  # Add batch dimension

    # Extract features
    with torch.no_grad():
        features = model.forward_features(image_tensor)

    return features.cpu().numpy()  # Assuming you want to work with NumPy arrays


In [5]:
feature = load_image_and_extract_features('18_em_1_0')

In [6]:
print(feature)

[[[ 1.7641672  -1.7546678  -1.3175337  ...  0.49328983  0.8133331
   -0.2152125 ]
  [ 1.6989812  -3.077386   -3.081486   ... -1.3945808   1.6221378
    0.52515376]
  [ 1.7413764  -2.6316273  -4.2409377  ... -0.8870623   1.7095534
    0.525716  ]
  ...
  [ 1.593117   -2.9912634  -2.3118224  ... -0.88721764  2.6196065
    0.29637212]
  [ 1.7858946  -3.2183542  -2.0843816  ... -1.001275    2.6518574
    0.2728033 ]
  [ 1.779844   -3.1831226  -1.3077934  ... -1.5635386   2.615067
   -0.02355908]]]


In [7]:
import json
temp_list = feature.tolist()  # Convert ndarray to a list

# Now, you can serialize temp_list
temp_string = json.dumps(temp_list)

temp_string
# feature_string = json.dumps(feature)


Output hidden; open in https://colab.research.google.com to view.

In [8]:
import numpy as np

def parse_and_flatten_array_string(array_string):
    # Remove the outer brackets and split the string into rows
    rows = array_string.strip('[]').split('\n')

    # Initialize an empty list to hold the parsed numbers
    parsed_numbers = []

    for row in rows:
        # Remove ellipsis and extra spaces, then strip inner brackets
        clean_row = row.replace('...', '').strip().strip('[]')

        # Split on spaces, and filter out any empty strings resulting from multiple spaces
        numbers = clean_row.split(' ')

        # Try converting each number to float and append to the list
        for num in numbers:
            try:
                parsed_numbers.append(float(num))
            except ValueError:
                # If conversion fails (e.g., due to empty strings after splitting), pass
                pass

    # Convert the list of numbers into a flattened numpy array
    flattened_array = np.array(parsed_numbers)

    # Convert the flattened array into a string
    array_string = ' '.join(map(str, flattened_array))

    return array_string


In [9]:
type(feature)

numpy.ndarray

In [10]:
string_representation = np.array2string(feature, separator=' ')
# Remove the brackets and newlines
string_representation = string_representation.replace('[', '').replace(']', '').replace('\n', '').strip()

print(string_representation)

1.7641672  -1.7546678  -1.3175337  ...  0.49328983  0.8133331   -0.2152125    1.6989812  -3.077386   -3.081486   ... -1.3945808   1.6221378    0.52515376   1.7413764  -2.6316273  -4.2409377  ... -0.8870623   1.7095534    0.525716    ...   1.593117   -2.9912634  -2.3118224  ... -0.88721764  2.6196065    0.29637212   1.7858946  -3.2183542  -2.0843816  ... -1.001275    2.6518574    0.2728033    1.779844   -3.1831226  -1.3077934  ... -1.5635386   2.615067   -0.02355908


In [11]:
# Change the string_representation

In [12]:
# feature_str = parse_and_flatten_array_string(feature)

In [13]:
gpt_input = string_representation +' <SEP>'

In [14]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer
model_path = '/content/drive/MyDrive/Major_Project_Final-Yr/myGpt'
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)



In [15]:

# Prepare your input text (you might need to preprocess it similar to how your training data was prepared)
# input_text = "FeatureVector: [2.5617244, -3.7776365, -1.4471102, ..., 0.98041683, -0.45592824] <SEP>"
input_text = gpt_input
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate predictions (you may want to adjust the generation parameters according to your needs)
output = model.generate(input_ids, max_length=1000, num_return_sequences=1)

# Decode the generated token IDs to text
predicted_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(predicted_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


1.7641672  -1.7546678  -1.3175337 ...  0.49328983  0.8133331   -0.2152125    1.6989812  -3.077386   -3.081486  ... -1.3945808   1.6221378    0.52515376   1.7413764  -2.6316273  -4.2409377 ... -0.8870623   1.7095534    0.525716   ...   1.593117   -2.9912634  -2.3118224 ... -0.88721764  2.6196065    0.29637212   1.7858946  -3.2183542  -2.0843816 ... -1.001275    2.6518574    0.2728033    1.779844   -3.1831226  -1.3077934 ... -1.5635386   2.615067   -0.02355908 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9087447 <SEP> 0.9

In [16]:
# Assume 'large_text' is your variable that contains the large text
large_text = predicted_text

# Find the position of 'LaTeX:' in the text
index = large_text.rfind('LaTeX:')

# Extract the content after 'LaTeX:'
if index != -1:  # Ensure that 'LaTeX:' was found in the string
    content_after_latex = large_text[index + len('LaTeX:'):].strip()  # .strip() removes any leading/trailing whitespace
    print(content_after_latex)
else:
    print("The term 'LaTeX:' was not found in the text.")


The term 'LaTeX:' was not found in the text.
